## Aula 01 - Introdução a ciência de dados

O objetivo deste material é:
- explicar conceitos sobre notebooks na plataforma Jupyter;
- conduzir o aluno na leitura de um arquivo de dados;
- explorar estatísticas e capacidades das bibliotecas numpy e pandas;
- realizar primeiro experimento de aprendizagem automática.

---

### Jupyter notebooks

A plataforma Jupyter permite executar códigos e anotar observações ou resultados, documentando todo o processo. É importante perceber que as células são executadas isoladas, porém com espaço compartilhado em memória. Uma vez criada determinada variável, independente da célula, tal variável ficará disponível para todas as outras células. A única exceção é quando a variável tiver escopo delimitado por função ou classe.

---

### Leitura de arquivos

Nesse material faremos a leitura de arquivos utilizando como fonte alguns arquivos csv. Nos arquivos apresentados aqui as colunas serão delimitadas por tabulação e os dados das etapas de treino e teste estão em arquivos separados. Após a leitura, o Python utiliza a biblioteca Pandas para converter os dados em DataFrames e disponibilizar funções que viabilizam um conjunto de operações importantes para manipulação de dados.

In [ ]:
import pandas as pd

train = pd.read_csv('train_empregados.csv', sep='\t')
test  = pd.read_csv('test_empregados.csv', sep='\t')

---

### Informações dos DataFrames

As informações do DataFrame podem ser visualizadas com a função info() e as dimensões podem ser acessadas utilizando a propriedade shape.

In [ ]:
print('Dimensões do treino:', train.shape)
print('Dimensões do teste:',  test.shape)
print()

train.info()

---

### Manipulando e corrigindo os tipos

As colunas iniciando com CR e o % Faltas são colunas numéricas, mas o Pandas não conseguiu interpretar isso devido ao formato. É necessário substituir a ',' por '.' e forçar a transformação em dados númericos.

In [ ]:
cols = train.columns
numeric_columns = []

## varre as colunas procurando as regras no nome

for c in cols:
    if 'CR' in c or '%' in c:
        numeric_columns.append(c)

## cria uma cópia para não precisa reimportar se algo der errado
## e para cada coluna entendida como numérica, aplica o replace e a transformação pd.to_numeric
        
new_train = train.copy()    
for c in numeric_columns: 
    new_train[c] = pd.to_numeric(train[c].str.replace(',','.'))
        
new_train.info()

---

### Replicando para o conjunto de teste

Toda transformação que acontece no conjunto de treino deve ser levada para o conjunto de teste a fim de manter consistência e o processamento não enfrentar nenhum tipo de erro.

In [ ]:
new_test = test.copy()    
for c in numeric_columns: 
    new_test[c] = pd.to_numeric(test[c].str.replace(',','.'))
        
new_test.info()

---

### Regras de classificação

O intuito desse conjunto de dados é verificar se a partir de algumas informações, um determinado estudante estará empregado ou não ao final do curso. No momento, não será aplicada nenhuma regra de aprendizagem automática, portanto o sistema será especialista: as regras serão dadas por humanos. Ainda assim, alguns conceitos se estendem para a aprendizagem automática quando chegar a hora.

Vamos supor que a regra que melhor descrimina os empregados dos não-empregados seja o CR do ciclo básico. Criaremos uma regra onde todos os alunos com CR > 0.6 estarão empregados, e os outros não. Para isso, precisamos criar uma coluna no DataFrame chamada "pred" (de predição) com o valor "Não" e atribuiremos "Sim" quando a regra for verdadeira.

In [ ]:
new_train['pred'] = 'Não'
indices_regra_predicao = new_train['CR CicloBasico'] > 0.6
new_train['pred'][indices_regra_predicao] = 'Sim'

In [ ]:
## Ou podemos ter uma regra mais complexa

new_train['pred'] = 'Não'
indices_regra_predicao = (new_train['CR CicloBasico'] > 0.6) & (new_train['Fez estágio?'] == 'Sim')
new_train['pred'][indices_regra_predicao] = 'Sim'

In [ ]:
## Podemos contar quantos sim e não tem na coluna predição e na coluna que queremos acertar

from collections import Counter
print(Counter(new_train['pred']))
print(Counter(new_train['Empregado?']))

---

### Verificando o desempenho da regra

Não basta saber que a quantidade de sim e não está correta, afinal eles podem estar alinhados apenas em quantidade, mas acontecendo em linhas diferentes. Nesse sentido, é necessário verificar se cada predição baseada na regra conferiu com o valor original da coluna 'Empregado?'. Isso será feito utilizando uma comparação entre as duas colunas. Toda vez que a linha tiver o mesmo valor nas duas colunas significará que a regra estava certa. Interessa somar o resultado dessa coluna que é feitas de 1 e 0, respectivamente, verdadeiro e falso.

A acurácia é calculada pela razão de acertos e total de linhas. Quando o total de acertos for igual ao número de linhas, indica que a regra acertou com perfeição todas as respostas.

In [ ]:
acertos = new_train['pred'] == new_train['Empregado?']
acertos.sum() / acertos.count()

---

### Sua vez

Agora você deve aplicar a regra observada em sala de aula e verificar qual a maior acurácia que consegue alcançar tanto no cojunto de treino quanto no conjunto de teste.